In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# Install ipywidgets inside the Pyodide runtime (only needed once per browser cache)
try:
    import ipywidgets  # noqa
except Exception:
    import piplite
    await piplite.install("ipywidgets")
    import ipywidgets  # noqa

import ipywidgets as widgets
from IPython.display import display


def shannon_entropy_bits(p: float) -> float:
    """Shannon entropy of a Bernoulli(p) variable in bits."""
    eps = 1e-12
    p = float(np.clip(p, eps, 1 - eps))
    q = 1 - p
    return -(p * np.log2(p) + q * np.log2(q))


def effective_modes(p: float) -> float:
    """Effective number of outcomes (a.k.a. 'effective modes')"""
    return 2 ** shannon_entropy_bits(p)


def make_plots(p: float) -> None:
    """Create the three-panel visualization inspired by the lab figure."""
    p = float(np.clip(p, 0.0, 1.0))
    q = 1 - p
    H = shannon_entropy_bits(p)
    Neff = 2 ** H

    ps = np.linspace(0, 1, 600)
    Hs = np.array([shannon_entropy_bits(x) for x in ps])
    Ns = 2 ** Hs

    fig = plt.figure(figsize=(11, 7.2))
    gs = fig.add_gridspec(2, 2, height_ratios=[1, 0.95], hspace=0.55, wspace=0.35)

    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, :])

    ax1.plot(ps, Hs, linewidth=1.5)
    ax1.set_xlabel("Probability of getting heads, p")
    ax1.set_ylabel("Entropy H(p) [bits]")
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1.05)
    ax1.grid(True, alpha=0.25)
    ax1.scatter([p], [H], s=40, zorder=3)

    ax2.plot(ps, Ns, linewidth=1.5)
    ax2.set_xlabel("Probability of getting heads, p")
    ax2.set_ylabel("Effective number of modes, $2^{H(p)}$")
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0.9, 2.1)
    ax2.grid(True, alpha=0.25)
    ax2.scatter([p], [Neff], s=40, zorder=3)

    ax3.bar(["Heads", "Tails"], [p, q])
    ax3.set_ylim(0, 1.05)
    ax3.set_ylabel("Probability")
    ax3.grid(True, axis='y', alpha=0.25)

    fig.suptitle(f"Entropy H = {H:.3f} bits, effective modes = {Neff:.3f}", fontsize=18, y=0.98)
    plt.show()


# ---------- Widget UI (put this at the end) ----------
p_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    description='p (Heads):',
    continuous_update=True,
    layout=widgets.Layout(width='520px')
)

out = widgets.Output()

def update_plot(change=None):
    with out:
        out.clear_output(wait=True)
        make_plots(p_slider.value)

p_slider.observe(update_plot, names='value')

display(widgets.VBox([
    widgets.HTML("<b>Adjust the coin bias</b>: p = P(Heads)"),
    p_slider,
    out
]))

update_plot()